In [1]:
# CELL 1: MODIFIED (Imports, Setup, Config with AdamW/Focal Loss Params)

# Imports and Setup
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa # <--- Import tensorflow-addons
from tensorflow_addons.losses import CategoricalFocalCrossentropy # <--- Import Focal Loss

SEED = 42
# Reproducibility
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# --- Distributed Strategy Setup --- (Should remain the same)
gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")

strategy = None
num_gpus = len(gpus)
if num_gpus > 1:
    print(f"Using MirroredStrategy for {num_gpus} GPUs.")
    strategy = tf.distribute.MirroredStrategy()
elif num_gpus == 1:
    print("Using single GPU.")
else:
    print("WARNING: No GPU detected. Training will be on CPU.")

# --- Calculate Global Batch Size --- (Keep previous calculation, should be fine with 30GB VRAM)
per_replica_batch_size = 32
if num_gpus == 0: # CPU case
    global_batch_size = per_replica_batch_size
    print(f"Using CPU Batch Size: {global_batch_size}")
else:
    global_batch_size = per_replica_batch_size * num_gpus
    print(f"Using Global Batch Size: {global_batch_size} ({per_replica_batch_size} per replica/GPU)")

# --- NEW: Configuration with AdamW and Focal Loss Params ---
IMG_SIZE = 224
N_CLASSES = 5
EPOCHS_PHASE1 = 30
EPOCHS_PHASE2 = 25
LR_PHASE1 = 1e-4       # Initial LR for AdamW Phase 1
WD_PHASE1 = 1e-4       # Weight Decay for AdamW Phase 1
LR_PHASE2 = 1e-5       # Fine-tuning LR for AdamW Phase 2
WD_PHASE2 = 1e-5       # Weight Decay for AdamW Phase 2 (often reduced)
FOCAL_ALPHA = 0.25     # Alpha parameter for Focal Loss (common value)
FOCAL_GAMMA = 2.0      # Gamma parameter for Focal Loss (common value)
PATIENCE_EARLY_STOPPING = 10
PATIENCE_REDUCE_LR = 5
MIN_LR = 1e-7

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.17.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're u

ModuleNotFoundError: No module named 'keras.src.engine'

In [ ]:
pip install tensorflow addon

In [ ]:
# CELL 1.5 (New Cell): Install tensorflow-addons
# Run this cell once at the beginning
!pip install tensorflow-addons -q

In [ ]:
# CELL 2: MODIFIED (Load Data, Plot Original Dist, Oversample, Plot New Dist)
# (Keep the version from the previous improvement that includes oversampling)

# Load the CSV file containing your metadata
df = pd.read_csv('/kaggle/input/diabetic-retinopathy-dataset-ben-graham-applied/train.csv')
print("Original Dataset shape:", df.shape)
print(df.head())

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Map diagnosis to class names
diagnosis_mapping = {
    0: 'No_DR', 1: 'Mild', 2: 'Moderate', 3: 'Severe', 4: 'Proliferative_DR'
}
df['class_name'] = df['diagnosis'].map(diagnosis_mapping)

# Plot ORIGINAL class distribution
plt.figure(figsize=(8,5))
sns.countplot(x='class_name', data=df, order=list(diagnosis_mapping.values()))
plt.title("Original Distribution of DR Classes")
plt.xlabel("Diagnosis")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- Simple Oversampling Strategy ---
counts = df['diagnosis'].value_counts()
majority_count = counts[0]
max_samples_per_minority = 5000 # Adjust cap if needed

dfs_balanced = [df[df['diagnosis'] == 0]]

for i in range(1, N_CLASSES):
    df_minority = df[df['diagnosis'] == i]
    current_count = len(df_minority)
    # Increase minority classes up to 5x or the cap, whichever is smaller
    target_count = min(max_samples_per_minority, current_count * 5)
    if current_count < target_count:
        df_minority_oversampled = df_minority.sample(n=target_count, replace=True, random_state=SEED)
        dfs_balanced.append(df_minority_oversampled)
        print(f"Oversampling class {i} ({diagnosis_mapping[i]}) from {current_count} to {target_count}")
    else:
         dfs_balanced.append(df_minority)
         print(f"Keeping class {i} ({diagnosis_mapping[i]}) at {current_count} samples (already >= target/cap)")


df_balanced = pd.concat(dfs_balanced).sample(frac=1, random_state=SEED).reset_index(drop=True) # Shuffle

print(f"\nShape after oversampling minority classes (<= {max_samples_per_minority}): {df_balanced.shape}")

# Plot BALANCED class distribution
plt.figure(figsize=(8,5))
sns.countplot(x='class_name', data=df_balanced, order=list(diagnosis_mapping.values()))
plt.title("Distribution After Oversampling (Minority Cap)")
plt.xlabel("Diagnosis")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Use the balanced dataframe from now on
df_original = df # Keep original for reference if needed
df = df_balanced

In [ ]:
# CELL 3: Prepare File Paths and Verify Existence
# (Keep the version from previous improvements - ensure BASE_DATA_DIR etc. are correct)

BASE_DATA_DIR = '/kaggle/input/diabetic-retinopathy-dataset-ben-graham-applied/DIABETIC_RETINOPATHY_DATASET_BELL_GRAHAM_PROCESSED/BELL_GRAHAM_SEGREGATED'
actual_folder_mapping = {
    0: 'No_DR', 1: 'Mild', 2: 'Moderate', 3: 'Severe', 4: 'Proliferative_DR'
}
FILE_EXTENSION = ".jpeg"

print("\nConstructing image paths...")
df['image_path'] = df.apply(
    lambda row: os.path.join(
        BASE_DATA_DIR,
        actual_folder_mapping.get(row['diagnosis'], 'UNKNOWN_FOLDER'),
        f"{row['id_code']}{FILE_EXTENSION}"
    ),
    axis=1
)
print("Image paths constructed.")

# Verify that image files exist (check a sample first for speed)
print("\nChecking a sample of image paths...")
if df.empty:
     print("DataFrame is empty, skipping path check.")
else:
    sample_paths = df['image_path'].sample(min(100, len(df)), random_state=SEED)
    missing_samples = sample_paths[~sample_paths.apply(os.path.exists)]
    if not missing_samples.empty:
        print(f"ERROR: Found missing images!")
        print(missing_samples.head())
        # Decide how to handle: raise error, or remove missing entries
        # Example: df = df[df['image_path'].apply(os.path.exists)].reset_index(drop=True)
        #          print(f"Removed missing files. New df shape: {df.shape}")
        raise FileNotFoundError("Missing image files found. Please check paths.")
    else:
        print("Sample image path check passed.")

# Create a string version of diagnosis for ImageDataGenerator
df['diagnosis_str'] = df['diagnosis'].astype(str)

print("\nExample constructed paths:")
print(df['image_path'].head())

In [ ]:
# CELL 4: Check Image Readable
# (Keep the version from previous improvements)

def check_image_readable(image_path):
    """Checks if an image can be read by OpenCV."""
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not read image at {image_path}")
        return False
    return True

# Test reading a sample image
if not df.empty:
    sample_path_to_check = df['image_path'].iloc[0]
    if check_image_readable(sample_path_to_check):
        print(f"Successfully read sample image: {sample_path_to_check}")
        # Display original image (as read from disk)
        orig_img = cv2.cvtColor(cv2.imread(sample_path_to_check), cv2.COLOR_BGR2RGB)
        plt.imshow(orig_img)
        plt.title("Sample Original Image (Read by CV2)")
        plt.axis('off')
        plt.show()
    else:
        print("Reading failed for the sample image.")
else:
    print("DataFrame is empty, cannot check sample image.")

# Verification of pixel range (manual check recommended)
# sample_img_check = cv2.imread(df['image_path'].iloc[0])
# if sample_img_check is not None:
#     print("Sample image pixel range (min, max):", np.min(sample_img_check), np.max(sample_img_check))
#     print("Sample image dtype:", sample_img_check.dtype)
# else:
#     print("Could not read sample image for pixel range check.")
# Confirming uint8 [0, 255] based on user input.

In [ ]:
# CELL 5: MODIFIED (Data Generators - Use Oversampled Data, Confirmed Rescale)

# Using rescale=1./255 based on uint8 [0, 255] input images.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    # vertical_flip=False, # Generally not used for retinas
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.2 # Reserve 20% of the oversampled training data for validation
)

test_datagen = ImageDataGenerator(rescale=1./255) # Only rescale for test

# Split the OVERSAMPLED data
train_df, test_df = train_test_split(
    df, # Use the balanced df
    test_size=0.2,
    stratify=df['diagnosis_str'], # Stratify on the balanced data
    random_state=SEED
)

print(f"Train DF shape: {train_df.shape}")
print(f"Test DF shape: {test_df.shape}")

# Create generators using the GLOBAL batch size for multi-GPU
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='diagnosis_str',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=global_batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED
)

validation_generator = train_datagen.flow_from_dataframe(
    train_df, # Use the same train_df for validation split
    x_col='image_path',
    y_col='diagnosis_str',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=global_batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False, # No need to shuffle validation
    seed=SEED
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='image_path',
    y_col='diagnosis_str',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=global_batch_size,
    class_mode='categorical',
    shuffle=False # NEVER shuffle test data
)

# Class weights (can still be used alongside oversampling/Focal Loss)
classes_train = train_df['diagnosis'].values
cw = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(classes_train),
    y=classes_train
)
class_weights = {i: cw_val for i, cw_val in enumerate(cw)}
print("\nClass weights (calculated on oversampled training data):")
print(class_weights)

print(f"\nFound {train_generator.samples} training samples.")
print(f"Found {validation_generator.samples} validation samples.")
print(f"Found {test_generator.samples} test samples.")

In [ ]:
# CELL 6: MODIFIED (Build Model, Compile Phase 1 with AdamW and Focal Loss)

def build_model(num_classes=N_CLASSES, img_size=IMG_SIZE, fine_tune=False, fine_tune_at_block=4):
    """Builds ResNet50V2 with simplified head. Fine-tuning unfreezes from specified block."""
    input_shape = (img_size, img_size, 3)
    base_model = applications.ResNet50V2(
        weights='imagenet', include_top=False, input_shape=input_shape
    )

    # Determine fine-tuning start layer index
    fine_tune_layer_name = f'conv{fine_tune_at_block}_block1_out'
    try:
        fine_tune_at_index = [i for i, layer in enumerate(base_model.layers) if layer.name == fine_tune_layer_name][0]
        print(f"Fine-tuning layer name target: {fine_tune_layer_name} (Index: {fine_tune_at_index})")
    except IndexError:
        print(f"Warning: Layer '{fine_tune_layer_name}' not found. Using default index 143 for ResNet50V2 Block 4.")
        fine_tune_at_index = 143 # Approx start of block 4

    if fine_tune:
        base_model.trainable = True
        for layer in base_model.layers[:fine_tune_at_index]:
            layer.trainable = False
        num_trainable_base = len(base_model.layers) - fine_tune_at_index
        print(f"Fine-tuning enabled: {num_trainable_base} trainable layers in base model (from index {fine_tune_at_index}).")
    else:
        base_model.trainable = False
        print("Fine-tuning disabled: Base model frozen.")

    # Simplified head
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(256, activation='relu'), # Smaller dense layer
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# --- Build and compile Phase 1 model inside strategy scope ---
if strategy:
    with strategy.scope():
        print("\nBuilding and compiling Phase 1 model within MirroredStrategy scope...")
        model = build_model(fine_tune=False) # Start with frozen base

        # --- Use AdamW Optimizer ---
        optimizer_ph1 = tfa.optimizers.AdamW(
            learning_rate=LR_PHASE1, weight_decay=WD_PHASE1
        )
        print(f"Using AdamW Optimizer for Phase 1: LR={LR_PHASE1}, WD={WD_PHASE1}")

        # --- Use Categorical Focal Loss ---
        loss_fn = CategoricalFocalCrossentropy(
            alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA
        )
        print(f"Using Categorical Focal Loss: Alpha={FOCAL_ALPHA}, Gamma={FOCAL_GAMMA}")

        model.compile(
            optimizer=optimizer_ph1,
            loss=loss_fn,
            metrics=['accuracy', tf.keras.metrics.AUC(name='auc')] # Add AUC
        )
else: # Single GPU or CPU
     print("\nBuilding and compiling Phase 1 model for single device...")
     model = build_model(fine_tune=False)
     optimizer_ph1 = tfa.optimizers.AdamW(learning_rate=LR_PHASE1, weight_decay=WD_PHASE1)
     loss_fn = CategoricalFocalCrossentropy(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)
     model.compile(
         optimizer=optimizer_ph1,
         loss=loss_fn,
         metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
     )

model.summary()

In [ ]:
# CELL 7: MODIFIED (Phase 1 Training - Adjusted Callbacks/Epochs)

# Callbacks for Phase 1
checkpoint_path_ph1 = 'model_phase1_best_adamw_focal.keras' # Updated checkpoint name
callbacks_phase1 = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE_EARLY_STOPPING, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=PATIENCE_REDUCE_LR, min_lr=MIN_LR, verbose=1),
    ModelCheckpoint(checkpoint_path_ph1, monitor='val_loss', save_best_only=True, verbose=1)
    # tf.keras.callbacks.TensorBoard(log_dir='./logs_phase1', histogram_freq=1) # Optional TensorBoard
]

print("\n--- Starting Phase 1 Training ---")
print(f"Epochs: {EPOCHS_PHASE1}, Batch Size (Global): {global_batch_size}")
print(f"Optimizer: AdamW (LR={LR_PHASE1}, WD={WD_PHASE1})")
print(f"Loss: Categorical Focal Loss (Alpha={FOCAL_ALPHA}, Gamma={FOCAL_GAMMA})")
print(f"Training samples: {train_generator.samples}, Validation samples: {validation_generator.samples}")

# Clear session potentially helpful before long training
# tf.keras.backend.clear_session()
# gc.collect()

history_phase1 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE1,
    validation_data=validation_generator,
    callbacks=callbacks_phase1,
    class_weight=class_weights # Still use weights - can complement Focal Loss/Oversampling
    # verbose=1 # Default is 1
)

# Plot training history (Phase 1)
print("\n--- Plotting Phase 1 History ---")
plt.figure(figsize=(18, 6))
# ... (plotting code remains the same as previous version) ...
plt.subplot(1, 3, 1)
plt.plot(history_phase1.history['loss'], label='Train Loss')
plt.plot(history_phase1.history['val_loss'], label='Val Loss')
plt.title("Phase 1 Loss")
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history_phase1.history['accuracy'], label='Train Accuracy')
plt.plot(history_phase1.history['val_accuracy'], label='Val Accuracy')
plt.title("Phase 1 Accuracy")
plt.legend()

plt.subplot(1, 3, 3)
if 'auc' in history_phase1.history:
    plt.plot(history_phase1.history['auc'], label='Train AUC')
    plt.plot(history_phase1.history['val_auc'], label='Val AUC')
    plt.title("Phase 1 AUC")
    plt.legend()

plt.tight_layout()
plt.show()

best_epoch_ph1 = np.argmin(history_phase1.history['val_loss']) + 1
print(f"Best Phase 1 epoch (based on val_loss): {best_epoch_ph1}")

# Optional cleanup after phase 1
# del history_phase1
# gc.collect()

In [ ]:
# CELL 8: MODIFIED (Phase 2 Fine-Tuning with AdamW and Focal Loss)

print(f"\n--- Preparing for Phase 2 Fine-Tuning ---")
print(f"Loading best weights from Phase 1: {checkpoint_path_ph1}")

# Load best weights into the *existing* model instance
model.load_weights(checkpoint_path_ph1)

# --- Modify the loaded model for fine-tuning within strategy scope ---
if strategy:
    with strategy.scope():
        print("Setting layers trainable and recompiling within strategy scope...")
        # Make base model layers trainable
        base_model = model.layers[0] # Get the base model from the Sequential container
        base_model.trainable = True
        fine_tune_at_block = 4 # Match the setting used in build_model definition
        fine_tune_layer_name = f'conv{fine_tune_at_block}_block1_out'
        try:
            fine_tune_at_index = [i for i, layer in enumerate(base_model.layers) if layer.name == fine_tune_layer_name][0]
        except IndexError:
            fine_tune_at_index = 143 # Fallback index

        for layer in base_model.layers[:fine_tune_at_index]:
            layer.trainable = False
        num_trainable_base = len(base_model.layers) - fine_tune_at_index
        print(f"Fine-tuning enabled: {num_trainable_base} trainable layers in base model (from index {fine_tune_at_index}).")

        # Re-compile with AdamW (lower LR/WD) and Focal Loss
        optimizer_ph2 = tfa.optimizers.AdamW(
            learning_rate=LR_PHASE2, weight_decay=WD_PHASE2
        )
        print(f"Using AdamW Optimizer for Phase 2: LR={LR_PHASE2}, WD={WD_PHASE2}")

        loss_fn = CategoricalFocalCrossentropy( # Use the same loss function instance/params
             alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA
        )
        print(f"Using Categorical Focal Loss: Alpha={FOCAL_ALPHA}, Gamma={FOCAL_GAMMA}")

        model.compile(
            optimizer=optimizer_ph2,
            loss=loss_fn,
            metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
        )
else: # Single GPU or CPU
     print("Setting layers trainable and recompiling for single device...")
     base_model = model.layers[0]
     base_model.trainable = True
     # ... (find fine_tune_at_index logic as above) ...
     try:
        fine_tune_at_index = [i for i, layer in enumerate(base_model.layers) if layer.name == f'conv{fine_tune_at_block}_block1_out'][0]
     except IndexError:
        fine_tune_at_index = 143
     for layer in base_model.layers[:fine_tune_at_index]:
         layer.trainable = False
     num_trainable_base = len(base_model.layers) - fine_tune_at_index
     print(f"Fine-tuning enabled: {num_trainable_base} trainable layers in base model (from index {fine_tune_at_index}).")

     optimizer_ph2 = tfa.optimizers.AdamW(learning_rate=LR_PHASE2, weight_decay=WD_PHASE2)
     loss_fn = CategoricalFocalCrossentropy(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)
     model.compile(
         optimizer=optimizer_ph2,
         loss=loss_fn,
         metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
     )


print("Model Summary after enabling fine-tuning:")
model.summary() # Summary now shows trainable base layers

# Callbacks for Phase 2
checkpoint_path_ph2 = 'model_phase2_best_adamw_focal_finetuned.keras' # Updated name
callbacks_phase2 = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE_EARLY_STOPPING, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=PATIENCE_REDUCE_LR, min_lr=MIN_LR, verbose=1),
    ModelCheckpoint(checkpoint_path_ph2, monitor='val_loss', save_best_only=True, verbose=1)
    # tf.keras.callbacks.TensorBoard(log_dir='./logs_phase2', histogram_freq=1) # Optional
]

print("\n--- Starting Phase 2 Fine-Tuning ---")
print(f"Epochs: {EPOCHS_PHASE2}, Batch Size (Global): {global_batch_size}")
print(f"Optimizer: AdamW (LR={LR_PHASE2}, WD={WD_PHASE2})")
print(f"Loss: Categorical Focal Loss (Alpha={FOCAL_ALPHA}, Gamma={FOCAL_GAMMA})")

history_phase2 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE2,
    validation_data=validation_generator,
    callbacks=callbacks_phase2,
    class_weight=class_weights, # Continue using class weights if desired
    initial_epoch=0 # Start fine-tuning epochs from 0
    # verbose=1
)


# Plot training history (Phase 2)
print("\n--- Plotting Phase 2 History ---")
plt.figure(figsize=(18, 6))
# ... (plotting code remains the same as previous version) ...
plt.subplot(1, 3, 1)
plt.plot(history_phase2.history['loss'], label='Train Loss')
plt.plot(history_phase2.history['val_loss'], label='Val Loss')
plt.title("Phase 2 Loss")
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history_phase2.history['accuracy'], label='Train Accuracy')
plt.plot(history_phase2.history['val_accuracy'], label='Val Accuracy')
plt.title("Phase 2 Accuracy")
plt.legend()

plt.subplot(1, 3, 3)
if 'auc' in history_phase2.history:
    plt.plot(history_phase2.history['auc'], label='Train AUC')
    plt.plot(history_phase2.history['val_auc'], label='Val AUC')
    plt.title("Phase 2 AUC")
    plt.legend()

plt.tight_layout()
plt.show()

best_epoch_ph2 = np.argmin(history_phase2.history['val_loss']) + 1
print(f"Best Phase 2 epoch (based on val_loss): {best_epoch_ph2}")

# Assign the final model (potentially restored to best weights by EarlyStopping)
model_final = model

# Optional cleanup
# del history_phase2
# gc.collect()

In [ ]:
# CELL 9: MODIFIED (Evaluation - Load correct checkpoint, use balanced accuracy)

print(f"\n--- Evaluating Best Fine-Tuned Model from Phase 2 ---")
# The 'model_final' variable should hold the best model if EarlyStopping(restore_best_weights=True) worked.
# Alternatively, explicitly load the best saved checkpoint for certainty.
print(f"Loading model from: {checkpoint_path_ph2}")

if strategy:
    with strategy.scope():
        # Load within scope is often safer
        model_to_evaluate = tf.keras.models.load_model(checkpoint_path_ph2)
else:
    model_to_evaluate = tf.keras.models.load_model(checkpoint_path_ph2)

print("\nEvaluating on Test Set...")
test_generator.reset() # Ensure generator starts from the beginning
results = model_to_evaluate.evaluate(test_generator, verbose=1)

print("\nTest Evaluation Results:")
print(f" - Test Loss: {results[0]:.4f}")
print(f" - Test Accuracy: {results[1]:.4f}")
if len(results) > 2:
    print(f" - Test AUC: {results[2]:.4f}") # Index depends on metrics order

print("\nGenerating Predictions on Test Set...")
test_generator.reset()
y_pred_probs = model_to_evaluate.predict(test_generator)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

# Map numerical class indices (0, 1, 2...) from generator to actual names
class_indices_map = {v: k for k, v in test_generator.class_indices.items()} # {0: '0', 1: '1', ...}
class_names_ordered = [diagnosis_mapping[int(class_indices_map[i])] for i in range(N_CLASSES)] # ['No_DR', 'Mild', ...]

print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names_ordered, digits=3))

bal_acc = balanced_accuracy_score(y_true, y_pred_classes)
print(f"\nBalanced Test Accuracy: {bal_acc:.4f}")

print("\nConfusion Matrix:")
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names_ordered, yticklabels=class_names_ordered)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
# CELL 10: MODIFIED (Single Image Prediction - Use final evaluated model)

# Use the model loaded for evaluation
# model_to_predict = tf.keras.models.load_model(checkpoint_path_ph2) # Or use this if needed
model_to_predict = model_to_evaluate # Use the model already loaded in Cell 9

def predict_single_image(image_path, model, img_size=IMG_SIZE):
    """Loads, preprocesses (matching generator), and predicts a single image."""
    try:
        img = tf.keras.preprocessing.image.load_img(
            image_path, target_size=(img_size, img_size)
        )
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        # Apply the SAME rescaling used in the generators
        img_array = img_array / 255.0

        img_batch = np.expand_dims(img_array, axis=0)
        predictions = model.predict(img_batch)
        predicted_class_index = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class_index]

        predicted_class_name = diagnosis_mapping[predicted_class_index]
        probabilities = {diagnosis_mapping[i]: prob for i, prob in enumerate(predictions[0])}

        return {
            'predicted_class': predicted_class_name,
            'confidence': confidence,
            'probabilities': probabilities,
            'processed_image_array': img_array
        }
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Test prediction on a sample test image
# Ensure test_df is available from Cell 5 split
if 'test_df' in locals() and not test_df.empty:
     sample_image_path = test_df['image_path'].iloc[random.randint(0, len(test_df)-1)] # Pick random test image
else: # Fallback if test_df not available
    sample_image_path ='/kaggle/input/diabetic-retinopathy-dataset-ben-graham-applied/DIABETIC_RETINOPATHY_DATASET_BELL_GRAHAM_PROCESSED/BELL_GRAHAM_SEGREGATED/Severe/1008_left.jpeg'

print(f"\n--- Predicting single image: {sample_image_path} ---")
result = predict_single_image(sample_image_path, model_to_predict)

if result:
    print(f"Predicted Class: {result['predicted_class']}")
    print(f"Confidence: {result['confidence']:.4f}")
    print("Class Probabilities:", result['probabilities'])

    # Display original and the array that went into the model
    plt.figure(figsize=(12,5))
    # ... (plotting code remains the same) ...
    plt.subplot(1,2,1)
    orig_img = cv2.cvtColor(cv2.imread(sample_image_path), cv2.COLOR_BGR2RGB)
    plt.imshow(orig_img)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(result['processed_image_array'])
    plt.title("Processed Image (as fed to model)")
    plt.axis('off')
    plt.show()
else:
    print("Prediction failed.")

In [ ]:
# CELL 11: MODIFIED (Save Final Model - Use the evaluated model)

# Save the final model (the one loaded/evaluated in Cell 9)
final_model_save_path = '/kaggle/working/diabetic_retinopathy_resnet50v2_adamw_focal_final.keras' # Updated name
print(f"\n--- Saving final model to {final_model_save_path} ---")
model_to_evaluate.save(final_model_save_path) # Save the model used for final evaluation
print("Model saved successfully.")

# Optional: Test loading the final saved model
# print("\nTesting loading the saved final model...")
# loaded_final_model = tf.keras.models.load_model(final_model_save_path)
# print("Model loaded successfully.")

print("\n--- Notebook execution finished ---")